**Задание 0. Загрузите необходимые библиотеки**

In [12]:
!pip install nltk spacy
!pip install pymystem3
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm
import spacy
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from pymystem3 import Mystem
import pandas as pd

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.8.0/ru_core_news_sm-3.8.0-py3-none-any.whl (15.3 MB)
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**Задание 1. Загрузка данных**

Вставьте текст для обработки согласно вашему варианту

In [14]:
text = """SpaceX планирует отправить первую пилотируемую миссию на Марс к 2029 году, несмотря на множество технических проблем.
Марсианский грунт содержит до 60% перхлоратов — токсичных соединений, представляющих опасность для человека.
Radiation на поверхности Красной планеты в 2,5 раза превышает допустимые нормы для астронавтов NASA!
"Мы разрабатываем специальные Martian habitats, защищающие от космической радиации и перепадов температур от +20°C до -153°C," — сообщает главный инженер проекта.
Одна из ключевых технологий — ISRU (In-Situ Resource Utilization), позволяющая производить кислород из CO₂ марсианской атмосферы.
Марсоход Perseverance успешно протестировал этот процесс в 2022 году, получив 12 г O₂ за час работы.
Российско-европейский проект ExoMars-2028 предполагает бурение на глубину до 2 м для поиска следов древней жизни.
По оценкам экспертов, первая самодостаточная колония на Марсе численностью 1000+ человек может появиться к 2050 году."""

**Задание 2. Нормализация текста.**

Приведите текст к нижнему регистру, удалите лишние пробоелы, переносы строк, спецсимволы, пунктуацию, обработайте цифры.

In [15]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[©®™℠]+', ' ', text)
    text = re.sub(r'\([^)]*\)', ' ', text)
    text = text.replace('ё', 'е')
    text = re.sub(r'(\w)-(\w)', r'\1HYPHEN\2', text)
    text = re.sub(r'[^\w\sHYPHEN]', ' ', text)
    text = re.sub(r'HYPHEN', '-', text)
    text = re.sub(r'\b\d+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

normalized_text = normalize_text(text)
print(normalized_text)

spacex планирует отправить первую пилотируемую миссию на марс к году несмотря на множество технических проблем марсианский грунт содержит до перхлоратов токсичных соединений представляющих опасность для человека radiation на поверхности красной планеты в раза превышает допустимые нормы для астронавтов nasa мы разрабатываем специальные martian habitats защищающие от космической радиации и перепадов температур от c до c сообщает главный инженер проекта одна из ключевых технологий isru позволяющая производить кислород из co₂ марсианской атмосферы марсоход perseverance успешно протестировал этот процесс в году получив г o₂ за час работы российско-европейский проект exomars- предполагает бурение на глубину до м для поиска следов древней жизни по оценкам экспертов первая самодостаточная колония на марсе численностью человек может появиться к году


**Задание 3. Токенизация**

Токенизируйте текст.

In [16]:
def tokenize(normalized_text):
    tokens = re.findall(r'\b[\w\-]+\b', normalized_text)

    return tokens

tokenized_text = tokenize(normalized_text)
print(tokenized_text)

['spacex', 'планирует', 'отправить', 'первую', 'пилотируемую', 'миссию', 'на', 'марс', 'к', 'году', 'несмотря', 'на', 'множество', 'технических', 'проблем', 'марсианский', 'грунт', 'содержит', 'до', 'перхлоратов', 'токсичных', 'соединений', 'представляющих', 'опасность', 'для', 'человека', 'radiation', 'на', 'поверхности', 'красной', 'планеты', 'в', 'раза', 'превышает', 'допустимые', 'нормы', 'для', 'астронавтов', 'nasa', 'мы', 'разрабатываем', 'специальные', 'martian', 'habitats', 'защищающие', 'от', 'космической', 'радиации', 'и', 'перепадов', 'температур', 'от', 'c', 'до', 'c', 'сообщает', 'главный', 'инженер', 'проекта', 'одна', 'из', 'ключевых', 'технологий', 'isru', 'позволяющая', 'производить', 'кислород', 'из', 'co₂', 'марсианской', 'атмосферы', 'марсоход', 'perseverance', 'успешно', 'протестировал', 'этот', 'процесс', 'в', 'году', 'получив', 'г', 'o₂', 'за', 'час', 'работы', 'российско-европейский', 'проект', 'exomars', 'предполагает', 'бурение', 'на', 'глубину', 'до', 'м', 'д

**Задание 4. Удаление стоп-слов**

Выведите 2 списка - 1. Очищенных токенов, 2. Список удаленных стоп-слов

In [18]:
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))

filtered_tokens = []
removed_stopwords = []

for token in tokenized_text:
    if token in russian_stopwords:
        removed_stopwords.append(token)
    else:
        filtered_tokens.append(token)

print(filtered_tokens)
print(removed_stopwords)

['spacex', 'планирует', 'отправить', 'первую', 'пилотируемую', 'миссию', 'марс', 'году', 'несмотря', 'множество', 'технических', 'проблем', 'марсианский', 'грунт', 'содержит', 'перхлоратов', 'токсичных', 'соединений', 'представляющих', 'опасность', 'человека', 'radiation', 'поверхности', 'красной', 'планеты', 'раза', 'превышает', 'допустимые', 'нормы', 'астронавтов', 'nasa', 'разрабатываем', 'специальные', 'martian', 'habitats', 'защищающие', 'космической', 'радиации', 'перепадов', 'температур', 'c', 'c', 'сообщает', 'главный', 'инженер', 'проекта', 'одна', 'ключевых', 'технологий', 'isru', 'позволяющая', 'производить', 'кислород', 'co₂', 'марсианской', 'атмосферы', 'марсоход', 'perseverance', 'успешно', 'протестировал', 'процесс', 'году', 'получив', 'г', 'o₂', 'час', 'работы', 'российско-европейский', 'проект', 'exomars', 'предполагает', 'бурение', 'глубину', 'м', 'поиска', 'следов', 'древней', 'жизни', 'оценкам', 'экспертов', 'первая', 'самодостаточная', 'колония', 'марсе', 'численно

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Задание 5. Лемматизация и стемминг**

Примените к токенам алгоритмы лемматизации и стемминга. Выведите 2 списка - 1. Лемматизированные токены 2. Стемматизированные токены

In [19]:
english_stemmer = SnowballStemmer("english")
russian_stemmer = SnowballStemmer("russian")
nltk.download('wordnet')

mystem = Mystem()
lemmatizer_en = WordNetLemmatizer()

def is_cyrillic(token):
    return any('а' <= char <= 'я' or char == 'ё' for char in token)

def is_latin(token):
    return any('a' <= char <= 'z' for char in token)

def stem_words(filtered_tokens):

    stemmed_tokens = []

    for token in filtered_tokens:
        if token.isalpha():
            if is_cyrillic(token):
                stem = russian_stemmer.stem(token)
                stemmed_tokens.append(stem)

            elif is_latin(token):
                stem = english_stemmer.stem(token)
                stemmed_tokens.append(stem)
    return stemmed_tokens

def lemm_words(filtered_tokens):

    lemmatized_tokens = []

    for token in filtered_tokens:
        if token.isalpha():
            if is_cyrillic(token):
                lemma = mystem.lemmatize(token)[0]
                lemmatized_tokens.append(lemma)

            elif is_latin(token):
                lemma = lemmatizer_en.lemmatize(token)
                lemmatized_tokens.append(lemma)
    return lemmatized_tokens

stemmed_tokens = stem_words(filtered_tokens)
lemmatized_tokens = lemm_words(filtered_tokens)

print('Стемматизированные токены: ', stemmed_tokens)
print('Лемматизированные токены: ', lemmatized_tokens)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Стемматизированные токены:  ['spacex', 'планир', 'отправ', 'перв', 'пилотируем', 'мисс', 'марс', 'год', 'несмотр', 'множеств', 'техническ', 'пробл', 'марсианск', 'грунт', 'содерж', 'перхлорат', 'токсичн', 'соединен', 'представля', 'опасн', 'человек', 'radiat', 'поверхн', 'красн', 'планет', 'раз', 'превыша', 'допустим', 'норм', 'астронавт', 'nasa', 'разрабатыва', 'специальн', 'martian', 'habitat', 'защища', 'космическ', 'радиац', 'перепад', 'температур', 'c', 'c', 'сообща', 'главн', 'инженер', 'проект', 'одн', 'ключев', 'технолог', 'isru', 'позволя', 'производ', 'кислород', 'марсианск', 'атмосфер', 'марсоход', 'persever', 'успешн', 'протестирова', 'процесс', 'год', 'получ', 'г', 'час', 'работ', 'проект', 'exomar', 'предполага', 'бурен', 'глубин', 'м', 'поиск', 'след', 'древн', 'жизн', 'оценк', 'эксперт', 'перв', 'самодостаточн', 'колон', 'марс', 'числен', 'человек', 'появ', 'год']
Лемматизированные токены:  ['spacex', 'планировать', 'отправлять', 'первый', 'пилотировать', 'миссия', 'мар

**Задание 6. Напишите функцию для препроцессинга текста**

Объедините все шаги в одну функцию. Выведите результат с лемматизированным списком

In [20]:
def is_cyrillic(token):
    return any('а' <= ch <= 'я' or ch == 'ё' for ch in token.lower())

def is_latin(token):
    return any('a' <= ch <= 'z' for ch in token.lower())

def big_function(text):

    text = text.lower()
    text = re.sub(r'[©®™℠]+', ' ', text)
    text = re.sub(r'\([^)]*\)', ' ', text)
    text = text.replace('ё', 'е')
    text = re.sub(r'(\w)-(\w)', r'\1HYPHEN\2', text)
    text = re.sub(r'[^\w\sHYPHEN]', ' ', text)
    text = re.sub(r'HYPHEN', '-', text)
    text = re.sub(r'\b\d+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = re.findall(r'\b[\w\-]+\b', text)

    filtered_tokens = [token for token in tokens if token not in russian_stopwords]

    stemmed_tokens = []
    for token in filtered_tokens:
        if token.isalpha():
            if is_cyrillic(token):
                stemmed_tokens.append(russian_stemmer.stem(token))
            elif is_latin(token):
                stemmed_tokens.append(english_stemmer.stem(token))

    lemmatized_tokens = []
    for token in filtered_tokens:
        if token.isalpha():
            if is_cyrillic(token):
                lemmatized_tokens.append(mystem.lemmatize(token)[0])
            elif is_latin(token):
                lemmatized_tokens.append(lemmatizer_en.lemmatize(token))

    return {
        'tokens': tokens,
        'filtered_tokens': filtered_tokens,
        'stemmed_tokens': stemmed_tokens,
        'lemmatized_tokens': lemmatized_tokens
    }

result = big_function(text)

print('Лемматизированные токены:', result['lemmatized_tokens'])

Лемматизированные токены: ['spacex', 'планировать', 'отправлять', 'первый', 'пилотировать', 'миссия', 'марс', 'год', 'несмотря', 'множество', 'технический', 'проблема', 'марсианский', 'грунт', 'содержать', 'перхлорат', 'токсичный', 'соединение', 'представлять', 'опасность', 'человек', 'radiation', 'поверхность', 'красный', 'планета', 'раз', 'превышать', 'допустимый', 'норма', 'астронавт', 'nasa', 'разрабатывать', 'специальный', 'martian', 'habitat', 'защищать', 'космический', 'радиация', 'перепад', 'температура', 'c', 'c', 'сообщать', 'главный', 'инженер', 'проект', 'один', 'ключевой', 'технология', 'isru', 'позволять', 'производить', 'кислород', 'марсианский', 'атмосфера', 'марсоход', 'perseverance', 'успешно', 'протестировать', 'процесс', 'год', 'получать', 'г', 'час', 'работа', 'проект', 'exomars', 'предполагать', 'бурение', 'глубина', 'м', 'поиск', 'след', 'древний', 'жизнь', 'оценка', 'эксперт', 'первый', 'самодостаточный', 'колония', 'марс', 'численность', 'человек', 'появляться'